# Ratterdam Beltway Decoding / ML approaches
## Mid August 2020 - Last attempts at finding a decoding method that works and we have confidence in the results
### Ideas: 
### 1) RF at a single alley per cell, so we can go back to 'template' approach that is invalid when using multiple alleys/cells
### 2) Cluster-based metrics compared to shuffle (not classification)
### 3) sliding window bayesian decoder (started this in another file, should dump what i've done here)

In [3]:
import sklearn as skl
from sklearn import svm, preprocessing, metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
from scipy import interp
from scipy.integrate import simps
from scipy.ndimage import center_of_mass
import numpy as np, random, json, pickle, datetime, copy, socket, os, sys, scipy
from scipy.stats import sem
import matplotlib.colors as colors
from importlib import reload

import utility_fx as util
import ratterdam_ParseBehavior as Parse
import ratterdam_CoreDataStructures as Core
import ratterdam_Defaults as Def
import ratterdam_DataFiltering as Filt

In [4]:
%matplotlib qt5
%qtconsole --style native

In [5]:
def createVisitSummaryFeatures(unit, alley, visit, features):
    """
    For a given pass for a given unit summarize the 1d ratemap into a simpler,
    explicit vector of attributes. Which attributes to use are given by
    the 'features' list. Visit is the visitnum not the data itself
    """
    feats = np.empty((0))
    rm = unit.alleys[alley][visit]['ratemap1d']
    # i dont know of a better way of doing this other than to just check param name in list and add it if present
    if 'rm' in features:
        feats = np.append(feats, rm)
    if 'time' in features:
        feats = np.append(feats, visit)
    if 'max95' in features:
        maximum = np.nanpercentile(rm, 95)
        feats = np.append(feats, maximum)
    if 'locmax95' in features:
        locmax = np.searchsorted(np.sort(rm), np.percentile(rm, 95))
        feats = np.append(feats, locmax)
    if 'mean' in features:
        mean = np.nanmean(rm)
        feats = np.append(feats, mean)
    if 'auc' in features:
        auc = simps(rm)
        feats = np.append(feats, auc)
    if 'avgdrds' in features:
        avgdrds = np.mean(np.abs(np.diff(rm))) # avg dr/ds change in rate / change in pos. 
        feats = np.append(feats, avgdrds)
    if 'maxdrds' in features:
        maxdrds = np.percentile(np.abs(np.diff(rm)), 95)
        feats = np.append(feats, maxdrds)
    if 'com' in features:
        try:
            com = center_of_mass(rm)[0]
            feats = np.append(feats, com)
        except:
            com = int(round(Def.singleAlleyBins[1]-1)/2)
            feats = np.append(feats, com)
    if 'comval' in features:
        try:
            comval = rm[int(np.round(com))]
            feats  = np.append(feats, comval)
        except:
            comval = np.nanpercentile(rm, 95)
            feats  = np.append(feats, comval)
    if 'boundMaxs' in features:
        # think there may be something going on at entrace/exit to alley so get the max val 
        # within each trisector of alley. NB real intersection ends with alleybounds_manuallyshifted2
        # gives a 3:6:3 ratio of approach L:alley:approach/exit R but I want to squeeze in the bounds to give
        # more space to the flanks (4:4:4 ratio) to capture whats happening at boundary itself as well.
        max1,max2,max3 = np.nanmax(rm[:4]), np.nanmax(rm[4:8]), np.nanmax(rm[9:]) # assumes 12bin rm. make generalized later
        feats = np.append(feats,(max1,max2,max3))
    return feats

In [6]:
# Load data into population dict. Each cell will be decoded separately. Within each cell each alley will be decoded separately.
rat = 'R808'
expCode = "BRD6"
datafile = f"E:\\Ratterdam\\{rat}\\{rat}{expCode}\\"

alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
population = {}
for subdir, dirs, fs in os.walk(datafile):
    for f in fs:
        if 'cl-maze1' in f and 'OLD' not in f and 'Undefined' not in f:
            clustname = subdir[subdir.index("TT"):] + "\\" + f
            unit = Core.UnitData(clustname, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
            unit.loadData_raw()
            rm = util.makeRM(unit.spikes, unit.position)            
            if np.nanpercentile(rm,Def.wholetrack_imshow_pct_cutoff) >= 1.:
                print(clustname)
                population[unit.name] = unit

E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:166: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:166: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:172: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:172: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWar

TT1\cl-maze1.2
TT12\cl-maze1.1
TT12\cl-maze1.3
TT15\cl-maze1.2
TT5\cl-maze1.5
TT6\cl-maze1.1
TT6\cl-maze1.2
TT6\cl-maze1.3


In [49]:
def setupAlleyData(unit, alley, repFx, features):
    """
    Create a matrix (n,b) where n is the number of trials at that alley 
    (usually with rewards removed, but that's done in the unit.loadRawData fx)
    and b are the number of spatial bins in the 1d ratemap of each trial at that alley
    """
    if features == 'rm':
        X = np.empty((0, Def.singleAlleyBins[0]-1))
    else:
        X = np.empty((0, len(features)))
    Y = np.empty((0))
    
    for visitNum,visit in enumerate(unit.alleys[alley]):
        reprm = repFx(unit, alley, visitNum, features)
        X = np.vstack((X, reprm))
        Y = np.append(Y, unit.alleys[alley][visitNum]['metadata']['stimulus'])
    
    X[np.where(~np.isfinite(X))] = 0
    X = preprocessing.StandardScaler().fit_transform(X)
    
    return X, Y

In [297]:
def runRandomForest(X, Y, parmdict):
    oobs = []
#     fimps = []
#     paths = []
    for i in range(parmdict['nRuns']):
        clf = RandomForestClassifier(n_estimators=parmdict['nTrees'], 
                                     oob_score=True,
                                     max_features = parmdict['Max Feats'],
                                     max_depth = parmdict['Max Depth']
                                    )       
        clf.fit(X,Y)
        oobs.append(clf.oob_score_)
#         fimps.append(clf.feature_importances_)
#         paths.append(clf.decision_path(X))
        
    return oobs

In [346]:
def parmString(parmdict, features):
    string = ''
    for k,v in parmdict.items():
        string += f"{k}:{v}\n"
    for f in features:
        string +=f"{f}\n"
    return string

In [298]:
parmdict = {
    'nRuns':25, # reps of decoding for a given dataset. tech replicates. 
    'nTrees':700, 
    'Max Depth':None, 
    'Max Feats':'auto',
    'Cell inclusion in population': '95pctile >=1Hz overall',
    'Visit inclusion in data matrix': '9x visits Mean alley activity >=1 Hz',
    'Bootstrap': 'None',
    'Shuffle': 1000
    }

features = [
    'time',
    'auc',
    'com',
    'comval',
    'max95',
    'locmax95',
    'mean',
    'avgdrds',
    'maxdrds'
           ]

In [ ]:
clust = 'TT15cl-maze1.2'
unit = population[clust]

string = parmString()
stamp = util.genTimestamp()

fig, axes = plt.subplots(3,3,figsize=(10,10))
plt.suptitle(f"{clust} RF Decoding by Alley")
plt.text(0.005, 0.2, string, fontsize=6, transform=plt.gcf().transFigure)

for i,alley in enumerate(Def.beltwayAlleys):
    valid = Filt.checkMinimumPassesActivity(unit, alley)
    if valid:
        ax = fig.axes[i]
        print(alley)
        X,Y = setupAlleyData(unit, alley, createVisitSummaryFeatures, features)
        realoobs = runRandomForest(X,Y, parmdict)
        realmean = np.mean(realoobs)
        nulloobs = np.zeros((0,1))
        for i in range(parmdict['Shuffle']):
            Ys = np.random.permutation(Y)
            ssoobs = runRandomForest(X,Ys, parmdict)
            nulloobs = np.vstack((nulloobs, np.mean(ssoobs)))
        
        ax.hist(nulloobs)
        ax.vlines(np.percentile(nulloobs, 95),0,150,'k')
        ax.vlines(realmean,0,100,'r')
        ax.set_title(f"Alley {alley}, mean {realmean} vs {np.percentile(nulloobs, 95)} 95% null pct")
    else:
        fig.axes[i].set_title(f"Insufficient Activity in Alley {alley} for Decoding")

16


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


## Cluster Metrics
#### Treat trials under a given texture as a cluster with a centroid. Use basic metrics like avg distance to centroid, intercentroid distance,
#### and intersample distance (compared to shuffle w corrected pvalue) to test effect of stimulus on neural representation

In [271]:
from sklearn import decomposition
from sklearn.neighbors import NearestCentroid
from sklearn import preprocessing
from scipy.spatial.distance import pdist
import numpy as np
import ratterdam_Defaults as Def

def interCentroidDistance(ncObj):
    """
    Input a NearestCentroid object (sklearn)
    and return pairwise Euclidian distances 
    between them
    """
    return pdist(ncObj.centroids_)

def avgDistToCentroid(ncObj,sa,sb,sc):
    """
    Input: ncObj - NearestCentroid object (sklearn)
           sa,sb,sc - vectors which produced centroids
    """
    avga = np.mean(np.linalg.norm(ncObj.centroids_[0]-sa,axis=1))
    avgb = np.mean(np.linalg.norm(ncObj.centroids_[1]-sb,axis=1))
    avgc = np.mean(np.linalg.norm(ncObj.centroids_[2]-sc,axis=1))
    return avga, avgb, avgc

def interSampleDistance(sa,sb,sc):
    """
    Input sa,sb,sc - samples from a,b,c trials
    Return avg Euclidian distance within each set of samples
    """
    avga = np.mean(pdist(sa))
    avgb = np.mean(pdist(sb))
    avgc = np.mean(pdist(sc))
    return avga, avgb, avgc
	
def setupData(unit, alley, features, dimred):
    """
    Input   - unit: Unit class object
            - alley: alley in whole track terms
            - list of features (e.g. COM, Max, etc) to represent a single RM
                or pass 'rm' to use original rm 
            - dimred: if False, no dimensionality reduction
                if a number then thats # pca comps

    Returns: data matrix X (n,f) n samples f features
             label vector Y (n,)
    """
    if 'rm' in features:
        atrials,btrials,ctrials = np.empty((0,Def.singleAlleyBins[0]-1)), np.empty((0,Def.singleAlleyBins[0]-1)), np.empty((0,Def.singleAlleyBins[0]-1))
    else:
        atrials,btrials,ctrials = np.empty((0,len(features))), np.empty((0,len(features))), np.empty((0,len(features)))
    for i,visit in enumerate(unit.alleys[alley]):
        stim = visit['metadata']['stimulus']
        feats = createVisitSummaryFeatures(unit,alley,i,features)
        if stim == 'A':
            atrials = np.vstack((atrials, feats))
        elif stim == 'B':
            btrials = np.vstack((btrials, feats))
        elif stim == 'C':
            ctrials = np.vstack((ctrials, feats))
            
    X = np.vstack((atrials, btrials, ctrials))
    X[np.where(~np.isfinite(X))] = 0
    X = preprocessing.StandardScaler().fit_transform(X)
    Y = np.asarray([0]*atrials.shape[0] + [1]*btrials.shape[0] + [2]*ctrials.shape[0])
    if dimred:
        if dimred > len(features) and 'rm' not in features:
            print("Error - PCA ncomp > original vector size")
        else:
            pca = decomposition.PCA(n_components=dimred)
            pca.fit(X)
            X = pca.transform(X)
    return X, Y

def splitSamplesbyLabel(X,y):
    """
    Given input matrix X with samples of different
    labels, stored in Y, split them into arrays by label
    A=0, B=1, C=2 for texture labels by convention
    """
    a = X[y==0]
    b = X[y==1]
    c = X[y==2]
    return a,b,c

def findCentroids(X,Y):
    """
    Given X (n,f) and Y (n,)
    create NearestCentroid object 
    and return it
    """
    nc = NearestCentroid(metric='euclidean')
    nc.fit(X,Y)
    return nc

def outlierRemoval(X,Y):
    """
    Removes outliers from whole data matrix X
    using IsolationForest (sklearn). 
    Returns X,Y with those samples removed
    """
    clf = IsolationForest()
    clf.fit(X)
    yo = clf.predict(X)
    X = X[yo==1]
    Y = Y[yo==1]
    return X,Y

In [485]:
# Intersample distance	
npca=3
features = ['rm']
X,Y = setupData(unit, alley, features, npca)
X,Y = outlierRemoval(X,Y)
cent = findCentroids(X,Y)
at,bt,ct = splitSamplesbyLabel(X,Y)
ad,bd,cd  = interSampleDistance(at,bt,ct)
mind = min(ad,bd,cd)

nshuff=1000
ss = np.empty((0))
for s in range(nshuff):
    Ys = np.random.permutation(Y)
    scent = findCentroids(X,Ys)
    sat,sbt,sct = splitSamplesbyLabel(X,Ys)
    ssd = np.min(interSampleDistance(sat,sbt,sct))
    ss = np.append(ss,ssd)
plt.figure()
plt.hist(ss)
plt.vlines(mind,0,200,'r')
plt.vlines(np.percentile(ss,1),0,200,'k')
plt.title(f"{unit.name} ISD pca comp = {npca}")


# Distance to centroids
npca=False
features = ['rm']
X,Y = setupData(unit, alley, features, npca)
X,Y = outlierRemoval(X,Y)
cent = findCentroids(X,Y)
at,bt,ct = splitSamplesbyLabel(X,Y)
ad,bd,cd  = avgDistToCentroid(cent,at,bt,ct)
mind = min(ad,bd,cd)

nshuff=1000
ss = np.empty((0))
for s in range(nshuff):
    Ys = np.random.permutation(Y)
    scent = findCentroids(X,Ys)
    sat,sbt,sct = splitSamplesbyLabel(X,Ys)
    ssdc = np.min(avgDistToCentroid(scent,sat,sbt,sct))
    ss = np.append(ss,ssdc)

plt.figure()
plt.hist(ss)
plt.vlines(mind,0,200,'r')
plt.vlines(np.percentile(ss,1),0,200,'k')
plt.title(f"{unit.name} MDC pca comp = {npca}")

# intercentroid distance
npca=False
features = ['rm']
X,Y = setupData(unit, alley, features, npca)
X,Y = outlierRemoval(X,Y)
cent = findCentroids(X,Y)
icd  = interCentroidDistance(cent)

nshuff=1000
ss = np.empty((0))
for s in range(nshuff):
    Ys = np.random.permutation(Y)
    scent = findCentroids(X,Ys)
    sicd = np.max(interCentroidDistance(scent))
    ss = np.append(ss,sicd)

plt.figure()
plt.hist(ss)
plt.vlines(max(icd),0,200,'r')
plt.vlines(np.percentile(ss,99),0,200,'k')
plt.title(f"{unit.name} ICD pca comp = {npca}")

Text(0.5,1,'TT1cl-maze1.2 ICD pca comp = False')

### Outlier Detection

In [443]:
a,b,c = splitSamplesbyLabel(X,Y)


In [444]:
# Running isolation forest on trials by txt, one each per loop
s = c
txt = 'C'

clf = IsolationForest()
clf.fit(s)
yo = clf.predict(s)
plt.figure()
plt.xlim([-2,6])
plt.ylim([-2,6])
plt.scatter(s[:,0], s[:,1])
plt.scatter(s[yo==-1][:,0], s[yo==-1][:,1],c='r')
plt.title(txt)

Text(0.5,1,'C')

In [445]:
# Running on all trials. seems like similar results and will do it this way for now
clf.fit(X)
yo = clf.predict(X)
plt.figure()
plt.xlim([-2,6])
plt.ylim([-2,6])
plt.title("All trials")
plt.scatter(X[:,0],X[:,1])
plt.scatter(X[yo==-1][:,0],X[yo==-1][:,1],c='r')

### Clustering Algs - Spectral Clustering Decoding, KMeans, 

In [495]:
stamp = util.genTimestamp()
parmdict = {
    'name':unit.name,
    'alley':5,
    'npca':False,
    'doSuffle':True,
    'nShuffle':500,
    'sliding':True,
    'ts':stamp,
    'alg':KMeans,
    'algParms':{'n_clusters':3},
    'metric':metrics.adjusted_rand_score,
    'nTrees':700,
    'nRuns':10,
    'Max Feats':None,
    'Max Depth':'auto'
            }

#features = ['com', 'max95', 'auc','locmax95', 'comval','mean']
features = ['rm']
string = parmString(parmdict, features)


X,Y = setupData(unit, parmdict['alley'], features, parmdict['npca'])
X,Y = outlierRemoval(X,Y)

if parmdict['sliding']:
    idx = slidingCArrWindow(X)
    allS = []
    allR = []
    for i in idx:
        Xw = X[:,i[0]:i[1]]
        model = parmdict['alg'](**parmdict['algParms'])
        l=model.fit_predict(Xw,Y)
        r = parmdict['metric'](Y,l)
        allR.append(r)

        if shuffle:
            s = []
            for i in range(nShuffle):
                s.append(parmdict['metric'](np.random.permutation(Y),l))
        allS.append(s)
        
# plt.figure()
# plt.text(0.005, 0.2, string, fontsize=6, transform=plt.gcf().transFigure)
# plt.hist(s)
# plt.vlines(r,0,200,'r')
# npct = np.percentile(s,95)
# plt.vlines(npct,0,150,'k')
# plt.title(f"{unit.name} A{alley} {parmdict['alg']}, ARI {round(r,2)} vs Shuffle 95% {round(npct,2)}")

In [388]:
def slidingCArrWindow(X,stepSize=1,winSize=4):
    """return col idx of arrays to slice in sliding window fashion"""
    _max = X.shape[1]
    idx = []
    for i in range(_max):
        a,b = 0+(i*stepSize), winSize+(i*stepSize)
        if b <= _max:
            idx.append([a,b])
    return idx

In [496]:
plt.plot([np.percentile(i,95) for i in allS],'k--')
plt.plot([min(i) for i in allS],'k--')
plt.plot(allR,'r')
plt.plot([np.percentile(i,99.2) for i in allS],'k')

### Sliding Window RF

In [500]:
stamp = util.genTimestamp()
parmdict = {
    'name':unit.name,
    'alley':5,
    'npca':False,
    'doSuffle':True,
    'nShuffle':100,
    'sliding':True,
    'ts':stamp,
    'alg':KMeans,
    'algParms':{'n_clusters':3},
    'metric':metrics.adjusted_rand_score,
    'nTrees':700,
    'nRuns':10,
    'Max Feats':None,
    'Max Depth':6
            }

#features = ['com', 'max95', 'auc','locmax95', 'comval','mean']
features = ['rm']
string = parmString(parmdict, features)

X,Y = setupData(unit, parmdict['alley'], features, parmdict['npca'])
X,Y = outlierRemoval(X,Y)

if parmdict['sliding']:
    idx = slidingCArrWindow(X)
    allS = []
    allR = []
    for i in idx:
        Xw = X[:,i[0]:i[1]]
        oob = np.mean(runRandomForest(Xw,Y,parmdict)) 
        
        if shuffle:
            s = []
            for i in range(parmdict['nShuffle']):
                s.append(np.mean(runRandomForest(Xw, np.random.permutation(Y), parmdict)))
            allS.append(s)